In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
import os
n_jobs = 16
os.environ["OMP_NUM_THREADS"] = str(n_jobs)
import joblib
import click
import json
import time
from glob import glob
import itertools
import collections.abc
import sys
from tqdm.notebook import tqdm
# !{sys.executable} -m pip install qcircuit
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import re
from joblib import Parallel, delayed, dump, load


# Qiskit
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Operator
from qiskit.primitives import StatevectorEstimator
from qiskit.circuit import Parameter, ParameterVector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_ibm_runtime import Batch
from qiskit_ibm_runtime import exceptions

In [2]:
def mitarai(quantumcircuit,num_wires,paramname='x'):
    # encoding as proposed by Mitarai et al.
    num_features = num_wires
    features = ParameterVector(paramname,num_features*2)
    for i in range(num_wires):
        feature_idx = i % num_features  # Calculate the feature index using modulo
        quantumcircuit.ry(np.arcsin(features[feature_idx * 2]), i)
        quantumcircuit.rz(np.arccos(features[feature_idx * 2 + 1] ** 2), i)


def double_angle(quantumcircuit, num_wires,paramname='x'):
    #  creates a circuit that encodes features into wires via angle encoding with an RY then RZ gate
    #  the features are encoded 1-1 onto the qubits
    #  if more wires are passed then features the remaining wires will be filled from the beginning of the feature list
    num_features = num_wires
    features = ParameterVector(paramname,num_features*2)
    for i in range(num_wires):
        feature_index = i % num_features
        quantumcircuit.ry(features[feature_index], i)
        quantumcircuit.rz(features[feature_index], i)

def entangle_cnot(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cnot gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cx(i, 0)
        else:
            quantumcircuit.cx(i, i+1)


def entangle_cz(quantumcircuit,num_wires):
    #  entangles all of the wires in a circular fashion using cz gates
    for i in range(num_wires):
        
        if i == num_wires - 1:
            quantumcircuit.cz(i, 0)
        else:
            quantumcircuit.cz(i, i+1)


def HardwareEfficient(quantumcircuit,num_wires,paramname='theta'):
    parameters = ParameterVector(paramname,num_wires*3)
    for qubit in range(num_wires):
        quantumcircuit.rx(parameters[qubit * 3], qubit)  
        quantumcircuit.rz(parameters[qubit * 3 + 1], qubit)  
        quantumcircuit.rx(parameters[qubit * 3 + 2], qubit)  
    entangle_cnot(quantumcircuit,num_wires)



In [3]:
def circuit(nqubits,RUD=1,AL=1):
    qc = QuantumCircuit(nqubits)
    for i in range(RUD):
        double_angle(qc,nqubits,paramname=f'x{i}')
        qc.barrier()
        for j in range(AL):        
            HardwareEfficient(qc,nqubits,paramname=f'theta{i}_{j}')
            qc.barrier()
    return qc

In [4]:
SMALL_SIZE = 8
MEDIUM_SIZE = 12
BIGGER_SIZE = 12
 
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

In [5]:
top=os.getcwd()
with open(os.path.join(top,'5_DDCC_train.bin'),'rb') as f:
    ddcc_train = joblib.load(f)

with open(os.path.join(top,'5_DDCC_test.bin'),'rb') as f:
    ddcc_test = joblib.load(f)

with open(os.path.join(top,'5_DDCC_scaler.bin'),'rb') as f:
    ddcc_scaler = joblib.load(f)

X_ddcc_train, y_ddcc_train = ddcc_train['X'],ddcc_train['y']
X_ddcc_test, y_ddcc_test = ddcc_test['X'],ddcc_test['y']

X_ddcc_train = X_ddcc_train.reshape(-1,64,5)
X_ddcc_test = X_ddcc_test.reshape(-1,64,5)
y_ddcc_train = y_ddcc_train.reshape(-1,64)
y_ddcc_test = y_ddcc_test.reshape(-1,64)


# X_train, y_train = X_ddcc_train, y_ddcc_train
# X_test, y_test = X_ddcc_test, y_ddcc_test
X_train, y_train = X_ddcc_train, y_ddcc_train
X_test, y_test = X_ddcc_test, y_ddcc_test

# X_train = [X_train[i:i+4] for i in range(0,len(X_train),4)]
# X_test = [X_test[i:i+4] for i in range(0,len(X_test),4)]
scaler = ddcc_scaler

# print(len(X_train),X_train[0].shape,X_train[-1].shape)
print(y_train.shape, y_test.shape)

(159, 64) (40, 64)


/Users/grierjones/miniconda3/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.5.1 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
def grab_data(path):
    '''
    Given a globbed path, return the job_n.txt, *_train.txt, and *_test.txt files
    '''
    iterations = []
    train_metric = []
    test_metric = []
    for i in path:
        try:
            numbers = int(re.search(r'jobs_(\d+)\.txt', i).group(1))
            iterations.append(i)
        except:
            if 'test' in i:
                test_metric.append(i)
            else:
                train_metric.append(i)
            
    return sorted(iterations,key=lambda x: int(re.search(r'jobs_(\d+)\.txt', x).group(1))), train_metric, test_metric

In [7]:
def fetch_jobs_from_file(service, filename):
    """Fetch all job results from a given file."""
    with open(filename, 'r') as f:
        job_ids = f.readlines()
    return [service.job(job_id.strip()).result() for job_id in job_ids]

def grab_jobs(lst):
    """Parallelize over job files, fetching results concurrently."""
    service = QiskitRuntimeService(
        channel='ibm_quantum',
        instance='pinq-quebec-hub/univ-toronto/default'
    )

    # Parallel processing of files
    jobs = Parallel(n_jobs=-1,backend='threading')(
        delayed(fetch_jobs_from_file)(service, filename) for filename in tqdm(lst)
    )

    return np.array(jobs)

In [8]:
# ran = grab_jobs(['jobs_0.txt'])

In [9]:
# y_pred = np.hstack([i.data.evs.flatten() for i in ran.flatten()])

In [10]:
# with open('y_pred.npy', 'wb') as f:
#     np.save(f, y_pred)


with open('y_pred.npy', 'rb') as f:
    y_pred = np.load(f)    

In [11]:
num_qubits = 5
RUD = 1
AL = 5

optimization_level = 2
shots = 3072
# shots = 1024.0 * 1
resilience_level = 1

optimization_level = int(optimization_level)
shots = int(shots)
# shots = 1024.0 * 1
resilience_level = int(resilience_level)

# 
qc = circuit(num_qubits,RUD,AL)

observables_labels = ''.join(['I']*(num_qubits-1))+"Z"

In [12]:
# params = [-2.90318345,2.23743464,-2.12427964,-0.11653103,0.55388708,-2.77010897
# ,3.06858498,2.18960145,1.18551998,-1.06648308,0.6027151,1.14432445
# ,1.31029899,-1.8735468,0.7296508,-2.90318345,2.23743464,-2.12427964
# ,-0.11653103,0.55388708,-2.77010897,3.06858498,2.18960145,1.18551998
# ,-1.06648308,0.6027151,1.14432445,1.31029899,-1.8735468,0.7296508
# ,-2.90318345,2.23743464,-2.12427964,-0.11653103,0.55388708,-2.77010897
# ,3.06858498,2.18960145,1.18551998,-1.06648308,0.6027151,1.14432445
# ,1.31029899,-1.8735468,0.7296508,-2.90318345,2.23743464,-2.12427964
# ,-0.11653103,0.55388708,-2.77010897,3.06858498,2.18960145,1.18551998
# ,-1.06648308,0.6027151,1.14432445,1.31029899,-1.8735468,0.7296508
# ,-2.90318345,2.23743464,-2.12427964,-0.11653103,0.55388708,-2.77010897
# ,3.06858498,2.18960145,1.18551998,-1.06648308,0.6027151,1.14432445
# ,1.31029899,-1.8735468,0.7296508,]

# with open('checkparams.bin','wb') as f:
#     joblib.dump({'x': params, 'loss': 0},f)

In [13]:
# from qregress import QiskitRegressor
from QiskitRegressor import QiskitRegressor
model = QiskitRegressor(qc,
                        num_qubits,
                        AL,
                        RUD,
                        'real', #'fake',
                        observables_labels,
                        channel='ibm_quantum',
                        instance='pinq-quebec-hub/univ-toronto/default',
                        parameterpath = 'checkparams.bin',
                        optimization_level = optimization_level,
                        resilience_level = resilience_level,
                        shots = shots,
                        iterations = 1,
                        verbose = True,
                        n_jobs = n_jobs)

checkparams.bin
Parameters loaded
[-2.90318345  2.23743464 -2.12427964 -0.11653103  0.55388708 -2.77010897
  3.06858498  2.18960145  1.18551998 -1.06648308  0.6027151   1.14432445
  1.31029899 -1.8735468   0.7296508  -2.90318345  2.23743464 -2.12427964
 -0.11653103  0.55388708 -2.77010897  3.06858498  2.18960145  1.18551998
 -1.06648308  0.6027151   1.14432445  1.31029899 -1.8735468   0.7296508
 -2.90318345  2.23743464 -2.12427964 -0.11653103  0.55388708 -2.77010897
  3.06858498  2.18960145  1.18551998 -1.06648308  0.6027151   1.14432445
  1.31029899 -1.8735468   0.7296508  -2.90318345  2.23743464 -2.12427964
 -0.11653103  0.55388708 -2.77010897  3.06858498  2.18960145  1.18551998
 -1.06648308  0.6027151   1.14432445  1.31029899 -1.8735468   0.7296508
 -2.90318345  2.23743464 -2.12427964 -0.11653103  0.55388708 -2.77010897
  3.06858498  2.18960145  1.18551998 -1.06648308  0.6027151   1.14432445
  1.31029899 -1.8735468   0.7296508 ]
[SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

In [15]:
y_predcheck = model.predict(X_train,restart=True,filename='og_jobs_0.txt')

Predicted in 275.7114 s


In [19]:
y_predcheck.shape

(10176,)

In [17]:
# y_predcheck_live = model.predict(X_train)
y_predcheck_live   = model.predict(X_train,restart=True,filename='jobs_None.txt')

Predicted in 275.2839 s


In [18]:
y_predcheck_live.shape

(10176,)

In [ ]:
261.0245 / 60

In [ ]:
y_predcheck.shape

In [ ]:
y_train.flatten().shape